# Moving adiabatically a ferromagnetic domain 

Here we consider moving a ferromagnetic chain occupying 3 sites of 4 sites chain form the left most position to the right most position. Contrary to what we do in the extending-retracting notebook, here we apply the adiabatic transformation resulting in the ferromagnetic area diaplcement to both end of the chain at the same time:
- we increase the Zeeman field on the first site to turn it in a paramagnetic site
- we decrease the Zeeman field on the last site to turn it in a ferromagnetic site

In [23]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from ising_kitaev import initialize_chain, move_chain, rotate_to_measurement_basis, add_measurement

## Setup

### Parameters

Here we define the strength of the on-site Zeeman for the ferromagnetic region and the paramagnetic region. Next we define the initially applied fields and the one in the final state.

In [24]:
zeeman_ferro = 0.01
zeeman_para = 10

In [25]:
initial_config = np.array([zeeman_ferro, zeeman_ferro, zeeman_ferro, zeeman_para])
final_config = np.array([zeeman_para, zeeman_ferro, zeeman_ferro, zeeman_ferro])

### Initialize the circuit

In [26]:
qreg = QuantumRegister(4)
creg = ClassicalRegister(3)
qcirc = QuantumCircuit(qreg, creg)

In [27]:
initialize_chain(qcirc, qreg, initial_config, 'logical_one')
qcirc.draw()

### Move the chain by one site

We evolve the Zeeman slowly and in between each update of the Zeeman we run the dynamic of the chain using Suzuki-Trotter expansion for t such that J*t = 1

In [28]:
move_chain(qcirc, qreg, initial_config, final_config, 0, 0.25, 0.25, 1, 5, method='both')
qcirc.depth()

1438

### Add the measurement

Before measuring we perform a change a basis to measure in the logical basis for the ferromagnetic chain (see README)

In [29]:
rotate_to_measurement_basis(qcirc, qreg, [1, 2, 3])
add_measurement(qcirc, qreg, creg, [1, 2, 3])

## Run the simulation

We run the simulation on the simulator since even in this case the depth of the circuit is already large.

In [30]:
from qiskit import Aer, execute

In [31]:
backend = Aer.get_backend('qasm_simulator')
job = execute(qcirc, backend, shots=2000)

In [32]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

We observe that we efficiently transferred our original state to the three site on the right with minimal leakage even though we use a coarse evolution description.

In [33]:
result = job.result()
print(result.get_counts())

{'000': 21, '001': 1956, '101': 23}
